In [2]:
import numpy as np
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [3]:
prices = []
brands = []
models = []
processors = []
RAM = []
ROM = []
os = []
ss = []
warranty = []

for i in range(1,21):
    print("https://www.flipkart.com/search?q=laptops&as=on&as-show=on&otracker=AS_Query_HistoryAutoSuggest_5_0_na_na_na&otracker1=AS_Query_HistoryAutoSuggest_5_0_na_na_na&as-pos=5&as-type=HISTORY&suggestionId=laptops&requestId=5d2ac1fd-a872-467d-b750-f81a9002504f&page="+str(i))
    urls = "https://www.flipkart.com/search?q=laptops&as=on&as-show=on&otracker=AS_Query_HistoryAutoSuggest_5_0_na_na_na&otracker1=AS_Query_HistoryAutoSuggest_5_0_na_na_na&as-pos=5&as-type=HISTORY&suggestionId=laptops&requestId=5d2ac1fd-a872-467d-b750-f81a9002504f&page="+str(i)
    
    page = requests.get(urls)
    soup = BeautifulSoup(page.text)

    p = soup.find_all("div", class_= "Nx9bqj _4b5DiR")
    for i in p:
        prices.append(i.text)
        
    data = soup.find_all("div", class_ = "KzDlHZ")
    for i in data:
        brands.append(i.text.split()[0])
    
    for i in data:
        text = i.text.strip()
        match = re.search(r'^\w+\s+(.*?)\s+(AMD|Intel|Apple|)', text)
        if match:
            models.append(match.group(1))  
   
    b = soup.find_all("li", class_="J+igdf")

    for i in b:
        m = i.text
        n = re.findall(r"(\d+)\sYear|Years",m)
        if len(n) > 0:
            warranty.append(n[0])

    product_count = len(soup.find_all("div", class_="tUxRFH"))  # Count of products per page
    current_index = 0
    while current_index < product_count:
        found = False
        for i in b[current_index * 5 : (current_index + 1) * 5]:  # Approx 5 <li> tags per product
            m = i.text
            match = re.findall(r"(Intel\s[\w\s]*?Core\s\w+|AMD\sRyzen\s\d+|Apple\sM\d(?:\sPro|\sMax)?|MediaTek\s[\w\s]+\d+)", m)
            if match:
                processors.append(match[0])
                found = True
                break
        if not found:
            processors.append("Unknown")
        current_index += 1
    
    c = soup.find_all("div", class_="KzDlHZ")
    for i in c:
        try:
            specs = i.text.split("-")[1].split("(")[1].split("/")
    
            ram_value = specs[0].split(" ")[0]
            rom_value = specs[1].split(" ")[0]
    
            RAM.append(ram_value)
            ROM.append(rom_value)
        except (IndexError, ValueError):
            RAM.append(None)
            ROM.append(None)
    
    d = soup.find_all("li", class_= "J+igdf")
    
    for i in d:
        m = i.get_text()
        n = re.findall(r"(\d+(?:\.\d+)?)\s?inch", m, re.IGNORECASE)
        if len(n) > 0:
            ss.append(n[0])
        
    product_blocks = soup.find_all("div", class_="tUxRFH")  # One block per product
    for block in product_blocks:
        lis = block.find_all("li", class_="J+igdf")
        found_os = None
        for li in lis:
            m = li.text
            match = re.search(r"(Windows\s\d+|macOS|Chrome(?:\sOS)?)", m, re.IGNORECASE)
            if match:
                found_os = match.group(0)
                break
        os.append(found_os if found_os else "Unknown")


https://www.flipkart.com/search?q=laptops&as=on&as-show=on&otracker=AS_Query_HistoryAutoSuggest_5_0_na_na_na&otracker1=AS_Query_HistoryAutoSuggest_5_0_na_na_na&as-pos=5&as-type=HISTORY&suggestionId=laptops&requestId=5d2ac1fd-a872-467d-b750-f81a9002504f&page=1
https://www.flipkart.com/search?q=laptops&as=on&as-show=on&otracker=AS_Query_HistoryAutoSuggest_5_0_na_na_na&otracker1=AS_Query_HistoryAutoSuggest_5_0_na_na_na&as-pos=5&as-type=HISTORY&suggestionId=laptops&requestId=5d2ac1fd-a872-467d-b750-f81a9002504f&page=2
https://www.flipkart.com/search?q=laptops&as=on&as-show=on&otracker=AS_Query_HistoryAutoSuggest_5_0_na_na_na&otracker1=AS_Query_HistoryAutoSuggest_5_0_na_na_na&as-pos=5&as-type=HISTORY&suggestionId=laptops&requestId=5d2ac1fd-a872-467d-b750-f81a9002504f&page=3
https://www.flipkart.com/search?q=laptops&as=on&as-show=on&otracker=AS_Query_HistoryAutoSuggest_5_0_na_na_na&otracker1=AS_Query_HistoryAutoSuggest_5_0_na_na_na&as-pos=5&as-type=HISTORY&suggestionId=laptops&requestId=5d2a

In [4]:
print("Prices:",len(prices))
print("Brands:",len(brands))
print("Models:",len(models))
print("Processors:",len(processors))
print("RAM:",len(RAM))
print("ROM:",len(ROM))
print("OS:",len(os))
print("SS:",len(ss))
print("Warranty:", len(warranty))

Prices: 480
Brands: 480
Models: 480
Processors: 480
RAM: 480
ROM: 480
OS: 480
SS: 481
Warranty: 436


In [5]:
f = ss[0:480]
len(f)

480

In [7]:
while len(warranty) < 480:
    warranty.append("1")

In [8]:
len(warranty)

480

In [9]:
d = {
"Brands":brands,
"Models": models,
"Prices":prices,
"Processors":processors,
"RAM":RAM,
"ROM":ROM,
"OS":os,
"Display":f,
"Warranty":warranty}

In [10]:
df = pd.DataFrame(d)

In [11]:
df

,Brands,Models,Prices,Processors,RAM,ROM,OS,Display,Warranty
0,Infinix,Y4,"₹34,990",Intel Core i5,16,512,Windows 11,16,1
1,MSI,Thin,"₹51,990",Intel Core i5,16,512,Windows 11,15.6,2
2,Acer,Aspire,"₹15,980",Intel Celeron Dual Core Processor,8,128,Windows 11,11.6,1
3,ASUS,Vivobook,"₹41,990",AMD Ryzen 3,8,512,Windows 11,15.6,1
4,Lenovo,IdeaPad,"₹39,990",Unknown,16,512,Windows 11,15.6,1
...,...,...,...,...,...,...,...,...,...
475,DELL,Intel,"₹59,740",Intel Core i5,8,1,Windows 11,14,1
476,ASUS,Vivobook,"₹40,990",Intel Core i7,8,512,Windows 11,14,1
477,HP,Intel,"₹51,490",Intel Core i7,16,512,Windows 11,15.6,1
478,DELL,Inspiron,"₹49,790",Intel Core i3,16,512,Windows 11,15.6,1


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Brands      480 non-null    object
 1   Models      480 non-null    object
 2   Prices      480 non-null    object
 3   Processors  480 non-null    object
 4   RAM         438 non-null    object
 5   ROM         438 non-null    object
 6   OS          480 non-null    object
 7   Display     480 non-null    object
 8   Warranty    480 non-null    object
dtypes: object(9)
memory usage: 33.9+ KB


In [14]:
df.to_csv(r"D:\Laptop_Sales.csv")

In [106]:
df["RAM"].value_counts()

RAM
16    211
8     171
4      11
24      7
12      5
32      5
Name: count, dtype: int64

In [107]:
df["OS"].value_counts()

OS
Windows 11    413
Chrome         13
Unknown         6
Name: count, dtype: int64

In [108]:
df[df["OS"] == "Unknown"]

,Brands,Models,Prices,Processors,RAM,ROM,OS,Display
145,Apple,MacBook,"₹1,25,990",Apple M3 Pro,16,512,Unknown,14
152,Apple,MacBook,"₹77,990",Intel Celeron Dual Core Processor,16,256,Unknown,11.6
173,HP,250,"₹23,390",AMD Ryzen 5,8,256,Unknown,14
185,Apple,MacBook,"₹1,63,990",Unknown,16,1,Unknown,15.6
200,Apple,Apple,"₹1,44,990",AMD Ryzen 5,24,512,Unknown,15.6
218,Primebook,Wifi,"₹13,490",MediaTek MT8183,4,64,Unknown,15.6


In [109]:
x = df[(df["OS"] == "Unknown") & (df["Brands"] == "Apple")]
print(x)

    Brands   Models     Prices                         Processors RAM  ROM  \
145  Apple  MacBook  ₹1,25,990                       Apple M3 Pro  16  512   
152  Apple  MacBook    ₹77,990  Intel Celeron Dual Core Processor  16  256   
185  Apple  MacBook  ₹1,63,990                            Unknown  16    1   
200  Apple    Apple  ₹1,44,990                        AMD Ryzen 5  24  512   

          OS Display  
145  Unknown      14  
152  Unknown    11.6  
185  Unknown    15.6  
200  Unknown    15.6  


In [110]:
df.loc[(df["Brands"] == "Apple") & (df["OS"].isin(["Unknown"])), "OS"] = "Mac OS"

In [111]:
df["OS"].value_counts()

OS
Windows 11    413
Chrome         13
Mac OS          4
Unknown         2
Name: count, dtype: int64

In [80]:
df[df["OS"] == "Unknown"]

,Brands,Models,Prices,Processors,RAM,ROM,OS,Display
173,HP,250,"₹23,390",AMD Ryzen 5,8,256,Unknown,14
218,Primebook,Wifi,"₹13,490",MediaTek MT8183,4,64,Unknown,15.6
